# 흰색 부분만 추출

In [2]:
import cv2
import numpy as np

image = cv2.imread('test/solidWhiteCurve.jpg')
mark = np.copy(image) # 복사

#bgr 제한값 설정
blue_threshold = 200
green_threshold = 200
red_threshold = 200

bgr_threshold = [blue_threshold, green_threshold, red_threshold]

#bgr 제한값보다 작으면 검은색으로
thredsholds = (image[:, :, 0] < bgr_threshold[0]) | (image[:, :, 1] < bgr_threshold[0]) | (image[:, :, 2] < bgr_threshold[0])
mark[thredsholds] = [0, 0, 0]

cv2.imshow('white', mark)
cv2.imshow('result', image)

cv2.waitKey(0)

13

# 사다리꼴에서 흰색부분 추출

In [2]:
import cv2
import numpy as np


def region_of_interest(img, vertices, color3 = (255, 255, 255), color1 = 255):
    #ROI setting
    mask = np.zeros_like(img) # img와 같은 크기의 빈 이미지
 
    if len(img.shape) > 2 : #if color image (3 channels)
        color = color3
    else : #if black&white image ( 1 channel)
        color = color1
    
    #vertices에 정한 점들로 이루어진 다각형 부분(ROI부분)을 color로 채움
    cv2.fillPoly(mask, vertices, color)
    
    #이미지와 color로 체워진 ROI를 합침
    ROI_img = cv2.bitwise_and(img, mask)
    return ROI_img

def mark_img(img, blue_threshold = 200, green_threshold = 200, red_threshold = 200 ):
    bgr_threshold = [blue_threshold, green_threshold, red_threshold]
    
    #bgr 제한값보다 작으면 검은색으로
    thredsholds = (image[:, :, 0] < bgr_threshold[0]) | (image[:, :, 1] < bgr_threshold[0]) | (image[:, :, 2] < bgr_threshold[0])
    
    mark[thredsholds] = [0, 0, 0]
    return mark 


image = cv2.imread('solidWhiteCurve.jpg')
height, width = image.shape[:2]

#사다리꼴 모형의 points
vertices = np.array([[(50, height), (width/2 - 45, height/2 + 60), (width/2 + 45, height/2 + 60), (width-50, height)]], dtype=np.int32)
roi_img = region_of_interest(image, vertices) # vertices에 정한 점들을 기준으로 ROI이미지 생
cv2.imshow('mid', roi_img)
mark = np.copy(roi_img)
mark = mark_img(roi_img)

cv2.imshow('roi_while', mark)
cv2.imshow('result', image)

cv2.waitKey(0)

13

# 동영상 적용

In [1]:
import cv2
import numpy as np


def region_of_interest(img, vertices, color3 = (255, 255, 255), color1 = 255):
    #ROI setting
    mask = np.zeros_like(img) # img와 같은 크기의 빈 이미지

    if len(img.shape) > 2 : #if color image (3 channels)
        color = color3
    else : #if black&white image ( 1 channel)
        color = color1
    
    #vertices에 정한 점들로 이루어진 다각형 부분(ROI부분)을 color로 채움
    cv2.fillPoly(mask, vertices, color)
    
    #이미지와 color로 체워진 ROI를 합침
    ROI_img = cv2.bitwise_and(img, mask)
    return ROI_img

def mark_img(img, blue_threshold = 200, green_threshold = 200, red_threshold = 200 ):
    bgr_threshold = [blue_threshold, green_threshold, red_threshold]
    
    #bgr 제한값보다 작으면 검은색으로
    thredsholds = (image[:, :, 0] < bgr_threshold[0]) | (image[:, :, 1] < bgr_threshold[1]) | (image[:, :, 2] < bgr_threshold[2])
    
    mark[thredsholds] = [0, 0, 0]
    return mark 

#----------------------------------------------------
#--------------------Load Video----------------------
cap = cv2.VideoCapture("solidWhiteRight.mp4")

while (cap.isOpened()):
    ret, image = cap.read()
    if not ret:
        break
    height, width = image.shape[:2]

    #사다리꼴 모형의 points
    vertices = np.array([[(50, height), (width/2 - 45, height/2 + 60), (width/2 + 45, height/2 + 60), (width-50, height)]], dtype=np.int32)
    roi_img = region_of_interest(image, vertices, (0, 0, 255)) # vertices에 정한 점들을 기준으로 ROI이미지 생성( 빨간색)

    mark = np.copy(roi_img)
    mark = mark_img(roi_img)

    #원본 이미지에 빨간색 인식 차선 overlap하기
    color_threshold = (mark[:,:,0] == 0) & (mark[:,:,1] == 0) & (mark[:,:,2] > 200)
    image[color_threshold] = [0,0,255]

    cv2.imshow('result', image) # 이미지 출력
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

# Release
cap.release()
cv2.destroyAllWindows()
#exit()